In [320]:
import midi2nmat as m2n
import pandas as pd
import numpy as np

In [321]:
path="/Users/wsdula/Documents/ATAVizM/Data/a-exsultate.mid"

In [322]:
test = m2n.midi2nmat(path)

In [323]:
def _make_ContCM(arr, deg1, deg2 = None):
    """
    Creates Continous Contour Matrix for searching
    Adapted for asynchrony
    """
    
    if deg2 is None:
        deg2 = deg1
    
    degrees = deg1 + deg2
    ContCM = np.zeros((degrees,len(arr))) #creates and initializes  contour matrix w/ zeroes
    ContCM.fill(np.nan)            #fills matrix with Null values
    #print(arr)
    for n in range(0,len(arr)):    
            for i in [1,2]:
                try:
                    if arr[n] > arr[n-i] and n-i >= 0 :
                        ContCM[deg1-i, n] = 1
                    elif n-i >= 0:
                        ContCM[deg1-i, n] = 0
                except:
                    continue

            for i in [1,2]:
                try:
                    if arr[n] > arr[n+i]:
                        ContCM[deg1+i-1, n] = 1
                    else:
                        ContCM[deg1+i-1, n] = 0
                except:
                    continue
    
    return ContCM

In [412]:
def _get_segments(cmin,cmax,pitches,CL_list):    
    p = []
    for n in range(cmin,cmax): #this should be a range of cardinalities to look over
        segments = [CL_list[0][i:i+n] for i in range(0, len(pitches)-n)] #get list of all possible segments of n cardinality
        segmentu = [list(x) for x in set(tuple(x) for x in segments)] #get the unique segments from the list

        ind = [[] for i in range(0,len(segmentu))] #get the index of these occurences of these segments, this also happens to be the index of the notes they start on
        for i in range(0,len(segmentu)):
            for j in range(0,len(segments)):
                if (segments[j] == segmentu[i]).all():
                    ind[i].append(j)

        if len(segments) == len(segmentu):
            break
        
        segmentucount = np.array([len(x) for x in ind]) #get a count of all the occurences for a given segment
#         segmentpitch = [pitches[ind[i][0]-2:ind[i][0]+n-1+2] for i in range(len(ind))] #get the pitches of the original occurence

        max_ind = np.argsort(segmentucount)[::-1][:5] #the indexes of the 5 most frequent segments so that the largest is in front
        max_segmentu = np.array(segmentu)[max_ind]
        max_segmentucount = segmentucount[max_ind]
#         max_segmentpitch = np.array(segmentpitch)[max_ind]
        max_segmentucov = [i*n for i in max_segmentucount]

        p.append({"segments":segments,
                  "seg":max_segmentu,
                  "segcount":max_segmentucount,
                  "segind":max_ind,
                  "segcov":max_segmentucov}) #storing the results in a data structure for later (will have index n-1)
    return p
    

In [448]:
def SAR_Contour(nmat, s_type=3, cmin=5, cmax=50, degrees=2):
    pitches=list(nmat['Pitch']) #Get the list of pitches for the piece
    contcom = _make_ContCM(pitches,degrees) #convert to continuous contour matrix
    CL = np.nansum(contcom, axis=0) #get the sums of the columns of the matrix
    
    if s_type > 1:
        durs=list(nmat['Time']) #extract beat onsets as a list
        durs=[x-y for x,y in zip(durs[1:len(durs)-1],durs[0:len(durs)-2])] #convert to interonset durations by subtracting adjacent onsets
        durs.append(nmat['Duration'].iloc[-1]) #add the duration of the last note back to list
        contcom=_make_ContCM(durs,degrees) #create continuous contour matrix
        DCL = np.nansum(contcom, axis=0) #get the sums of the columns of the matrix
        
    if s_type == 1:
        CL_list = [CL, CL]
    elif s_type == 2:
        CL_list = [DCL, DCL]
    elif s_type == 3:
        CL_list = [CL,DCL]
    
    p = _get_segments(cmin,cmax,pitches,CL_list) #makes a dataframe of all the important segments needed
    
    #initialize storage variables
#     allcov = []
    r = pd.DataFrame(columns=['card','seg','segcount','segind','cov','invind','inds'])
    
    for i in range(0,len(p)):
        for j in range(0,len(p[i]["seg"])):
            r = r.append({"card":i+cmin,
                  "seg":pitches[p[i]["segind"][j]:p[i]["segind"][j]+i+cmin],
                  "segcount":p[i]["segcount"][j],
                  "segind":p[i]["segind"][j],
                  "cov":p[i]["segcov"][j],
                  "invind":[],
                  "inds":list(range(p[i]["segind"][j],p[i]["segind"][j]+i+cmin+1))}, ignore_index=True)
#             allcov.append(p[i]["segcov"][j])
        
    r = r.sort_values(by=["cov"], ascending=False)
    
    return {"r":r, "p":p, "CL":CL_list, "nmat":nmat}

In [449]:
tuh = SAR_Contour(test)

In [450]:
tuh["r"]

,card,seg,segcount,segind,cov,invind,inds
160,37,"[67, 69, 70, 72, 70, 67, 69, 70, 69, 70, 69, 6...",3,121,111,[],"[121, 122, 123, 124, 125, 126, 127, 128, 129, ..."
155,36,"[65, 64, 64, 65, 67, 69, 69, 67, 69, 65, 62, 6...",3,85,108,[],"[85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 9..."
156,36,"[70, 69, 67, 67, 69, 65, 64, 65, 67, 67, 69, 6...",3,67,108,[],"[67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 7..."
150,35,"[64, 62, 64, 65, 67, 65, 60, 65, 65, 65, 62, 6...",3,25,105,[],"[25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 3..."
151,35,"[67, 70, 67, 69, 70, 69, 67, 67, 69, 65, 64, 6...",3,63,105,[],"[63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 7..."
152,35,"[67, 67, 69, 65, 67, 65, 64, 65, 64, 64, 65, 6...",3,75,105,[],"[75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 8..."
148,34,"[70, 69, 67, 67, 69, 65, 64, 65, 67, 67, 69, 6...",3,67,102,[],"[67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 7..."
146,34,"[69, 70, 69, 67, 69, 67, 67, 67, 70, 67, 69, 7...",3,129,102,[],"[129, 130, 131, 132, 133, 134, 135, 136, 137, ..."
145,34,"[69, 67, 67, 67, 70, 67, 69, 70, 69, 70, 69, 6...",3,133,102,[],"[133, 134, 135, 136, 137, 138, 139, 140, 141, ..."
147,34,"[69, 67, 69, 70, 67, 64, 62, 64, 65, 67, 65, 6...",3,20,102,[],"[20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 3..."
